<a href="https://colab.research.google.com/github/CagataySencan/StressAnalysis/blob/main/Reddit_Stress_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Suppor Vector Machine


In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import  GaussianNB
from sklearn.metrics import  confusion_matrix
from sklearn.preprocessing import LabelEncoder
from gensim.parsing.preprocessing import remove_stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn import metrics
import seaborn as sb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download ruchi798/stress-analysis-in-social-media

In [ ]:
! unzip stress-analysis-in-social-media.zip

In [ ]:
trainData = pd.read_csv("dreaddit-train.csv")
testData = pd.read_csv(("dreaddit-test.csv"))

In [ ]:
# Eksik veri kontrolü
print("-Eğitim verisi-\n",trainData.isnull().sum(),"\n-------------\n","-Test Verisi-\n",testData.isnull().sum())

-Eğitim verisi-
 subreddit                   0
post_id                     0
sentence_range              0
text                        0
id                          0
                           ..
lex_dal_avg_pleasantness    0
social_upvote_ratio         0
social_num_comments         0
syntax_fk_grade             0
sentiment                   0
Length: 116, dtype: int64 
-------------
 -Test Verisi-
 id                          0
subreddit                   0
post_id                     0
sentence_range              0
text                        0
                           ..
lex_dal_avg_pleasantness    0
social_upvote_ratio         0
social_num_comments         0
syntax_fk_grade             0
sentiment                   0
Length: 116, dtype: int64


In [ ]:
# Train ve test datalarımızı birleştirdik
X_Train = pd.DataFrame(trainData['text'], columns = ['text'])
X_Test = pd.DataFrame(testData['text'], columns = ['text'])
X = pd.concat((X_Train, X_Test), sort=False).reset_index(drop=True)

Y_Train =  pd.DataFrame(trainData['subreddit'], columns = ['subreddit'])
Y_Test =  pd.DataFrame(testData['subreddit'], columns = ['subreddit'])
Y =  pd.concat((Y_Train, Y_Test), sort=False).reset_index(drop=True)
X.head()

,text
0,"He said he had not felt that way before, sugge..."
1,"Hey there r/assistance, Not sure if this is th..."
2,My mom then hit me with the newspaper and it s...
3,"until i met my new boyfriend, he is amazing, h..."
4,October is Domestic Violence Awareness Month a...


In [ ]:
# Önişleme
nltk.download('wordnet')


In [ ]:
# Küçük harfe çevirme, stopwordslerden ve noktalama işaretlerinden temizleme
for i in range(X['text'].count()) : 
   
   text = X['text'][i]
   text = text.lower()
   text = remove_stopwords(text)
   text = text.translate(str.maketrans('','', punctuation))
   X['text'][i] = text

# Common wordlerden temizleme
cnt = Counter()
for text in X['text'].values : 
  for word in text.split() :
    cnt[word] += 1


X.head()



,text
0,said felt way before suggeted rest trigger ahe...
1,hey rassistance sure right place post this goe...
2,mom hit newspaper shocked this knows dont like...
3,met new boyfriend amazing kind sweet good stud...
4,october domestic violence awareness month dome...


In [ ]:
freq = set([w for (w, wc) in cnt.most_common(10)])
def removeFreqwords(text):
    return " ".join([word for word in str(text).split() if word not in freq])

X['text'] = X['text'].apply(lambda text: removeFreqwords(text))
X.head()

,text
0,said felt way before suggeted rest trigger ahe...
1,hey rassistance sure right place post this goe...
2,mom hit newspaper shocked this knows play hitt...
3,met new boyfriend amazing kind sweet good stud...
4,october domestic violence awareness month dome...


In [ ]:
# Lemmalama işlemi

lm = WordNetLemmatizer()

def lemmatizer(text):
    return " ".join([lm.lemmatize(word) for word in text.split()])

X['text'] = X['text'].apply(lambda text: lemmatizer(text))
X.head()


,text
0,said felt way before suggeted rest trigger ahe...
1,hey rassistance sure right place post this go ...
2,mom hit newspaper shocked this know play hitti...
3,met new boyfriend amazing kind sweet good stud...
4,october domestic violence awareness month dome...


In [ ]:
# Label Encoder ile subreddit label oluşturma
le = LabelEncoder()
subreddit_id = pd.DataFrame(le.fit_transform(Y), columns = ['subreddit_id'])
X['subreddit_id'] = subreddit_id
X.head()

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,text,subreddit_id
0,said felt way before suggeted rest trigger ahe...,6
1,hey rassistance sure right place post this go ...,2
2,mom hit newspaper shocked this know play hitti...,6
3,met new boyfriend amazing kind sweet good stud...,7
4,october domestic violence awareness month dome...,9


In [ ]:
# Vektörleme işlemi
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words='english')

features = tfidf.fit_transform(X.text).toarray()

labels = X.subreddit_id
X.head()

,text,subreddit_id
0,said felt way before suggeted rest trigger ahe...,6
1,hey rassistance sure right place post this go ...,2
2,mom hit newspaper shocked this know play hitti...,6
3,met new boyfriend amazing kind sweet good stud...,7
4,october domestic violence awareness month dome...,9


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25,random_state = 20)

In [ ]:
model = LinearSVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Karmaşıklık matrisi :\n", confusion_matrix(y_test, y_pred))

# Accuracy
from sklearn.metrics import accuracy_score
print("Doğruluk skoru : \n", accuracy_score(y_test, y_pred))

# Recall
from sklearn.metrics import recall_score
print("Duyarlılık skoru : \n",recall_score(y_test, y_pred, average=None))

# Precision
from sklearn.metrics import precision_score
print("Kesinlik skoru : \n",precision_score(y_test, y_pred, average=None))

Karmaşıklık matrisi :
 [[  4   3   8   1   0   6   7   3   0   0]
 [  0  97   8   8   0   2  25  23   0   4]
 [  2   5  48   4   0   5   5  10   0   1]
 [  2   3   7  31   0   0  26  19   1   6]
 [  0   0   2   0   3   1   1   0   0   0]
 [  2   4   5   3   0  28   5   5   0   3]
 [  0  23   4   6   0   2 100  23   0   6]
 [  0  17   4  11   0   0  19 131   1   3]
 [  1   7   4   0   0   0   5   6   2   3]
 [  0   1   4   8   0   0  20  21   1  20]]
Doğruluk skoru : 
 0.5219347581552306
Duyarlılık skoru : 
 [0.125      0.58083832 0.6        0.32631579 0.42857143 0.50909091
 0.6097561  0.70430108 0.07142857 0.26666667]
Kesinlik skoru : 
 [0.36363636 0.60625    0.5106383  0.43055556 1.         0.63636364
 0.46948357 0.54356846 0.4        0.43478261]


In [39]:
knn = KNeighborsClassifier(n_neighbors=50,metric="minkowski")
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
cm = confusion_matrix(y_test,y_pred)
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Karmaşıklık matrisi :\n", confusion_matrix(y_test, y_pred))

# Accuracy
from sklearn.metrics import accuracy_score
print("Doğruluk skoru : \n", accuracy_score(y_test, y_pred))

# Recall
from sklearn.metrics import recall_score
print("Duyarlılık skoru : \n",recall_score(y_test, y_pred, average=None))

# Precision
from sklearn.metrics import precision_score
print("Kesinlik skoru : \n",precision_score(y_test, y_pred, average=None))

Karmaşıklık matrisi :
 [[ 0  9  2  0  0  2  3  2  0  0]
 [ 0 73  0  0  0  0 14  4  0  0]
 [ 0 12  9  0  0  2  7  0  0  0]
 [ 0 22  1 12  0  0 23 11  0  1]
 [ 0  3  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  0  3  3  0  0  0]
 [ 0 40  0  4  0  0 54  6  0  1]
 [ 0 25  1  3  0  0 19 43  0  0]
 [ 0  9  0  0  0  0  3  1  0  0]
 [ 0  5  0  4  0  0 17  5  0  0]]
Doğruluk skoru : 
 0.4172043010752688
Duyarlılık skoru : 
 [0.         0.8021978  0.3        0.17142857 0.         0.23076923
 0.51428571 0.47252747 0.         0.        ]
Kesinlik skoru : 
 [0.         0.35609756 0.69230769 0.52173913 0.         0.42857143
 0.37762238 0.59722222 0.         0.        ]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
gnb = GaussianNB()
gnb.fit(x_train,y_train)
y_pred = gnb.predict(x_test)
cm = confusion_matrix(y_test,y_pred)
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Karmaşıklık matrisi :\n", confusion_matrix(y_test, y_pred))

# Accuracy
from sklearn.metrics import accuracy_score
print("Doğruluk skoru : \n", accuracy_score(y_test, y_pred))

# Recall
from sklearn.metrics import recall_score
print("Duyarlılık skoru : \n",recall_score(y_test, y_pred, average=None))

# Precision
from sklearn.metrics import precision_score
print("Kesinlik skoru : \n",precision_score(y_test, y_pred, average=None))

Karmaşıklık matrisi :
 [[ 0  5  2  2  0  1  6  2  0  0]
 [ 0 53  2  3  0  1 22 10  0  0]
 [ 2  8  6  1  0  1  9  3  0  0]
 [ 0 17  0 14  0  0 25 12  0  2]
 [ 0  1  0  1  0  0  1  0  0  0]
 [ 0  2  0  5  0  0  6  0  0  0]
 [ 0 32  0 12  0  0 41 16  0  4]
 [ 0 25  1 13  0  1 15 34  0  2]
 [ 0  8  0  1  0  0  4  0  0  0]
 [ 0  7  0  3  0  0 15  4  0  2]]
Doğruluk skoru : 
 0.3225806451612903
Duyarlılık skoru : 
 [0.         0.58241758 0.2        0.2        0.         0.
 0.39047619 0.37362637 0.         0.06451613]
Kesinlik skoru : 
 [0.         0.33544304 0.54545455 0.25454545 0.         0.
 0.28472222 0.41975309 0.         0.2       ]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
